In [13]:
# import pandas as pd
# import numpy as np
# import joblib

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder, StandardScaler
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline

# from sklearn.metrics import mean_squared_error, r2_score

# # Models
# from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import (
#     RandomForestRegressor, AdaBoostRegressor,
#     GradientBoostingRegressor, BaggingRegressor
# )
# from sklearn.svm import SVR
# from sklearn.neighbors import KNeighborsRegressor

# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor



# # ---------------------------------------------------
# # Adjusted R² Function
# # ---------------------------------------------------
# def adjusted_r2(r2, n, p):
#     return 1 - (1 - r2) * (n - 1) / (n - p - 1)


# # ---------------------------------------------------
# # Load Dataset
# # ---------------------------------------------------
# df = pd.read_csv("bollywood_movie_data.csv")

# print("Dataset Shape:", df.shape)

# df.drop("Movie Name", axis=1, inplace=True)
# df.drop("Release Period", axis=1, inplace=True)
# df.drop("New Director", axis=1, inplace=True)
# df.drop("Whether Franchise", axis=1, inplace=True)
# TARGET = "Revenue(INR)"
# y = df[TARGET]
# X = df.drop(TARGET, axis=1)


# # ---------------------------------------------------
# # Identify Columns
# # ---------------------------------------------------
# cat_cols = X.select_dtypes(include="object").columns.tolist()
# num_cols = X.select_dtypes(exclude="object").columns.tolist()

# print("Categorical Columns:", len(cat_cols))
# print("Numeric Columns:", len(num_cols))


# # ---------------------------------------------------
# # Reduce High Cardinality Columns
# # ---------------------------------------------------
# top_categories = {}
# def reduce_cardinality(df, col, top_k=15):
#     top = df[col].value_counts().nlargest(top_k).index.tolist()
#     top_categories[col] = top
#     df[col] = df[col].apply(lambda x: x if x in top else "Other")
#     return df


# high_card_cols = ["Movie Name", "Lead Star", "Director", "Music Director"]

# for col in high_card_cols:
#     if col in X.columns:
#         X = reduce_cardinality(X, col)


# # ---------------------------------------------------
# # Train-Test Split
# # ---------------------------------------------------
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )


# # ---------------------------------------------------
# # Models List (User Provided)
# # ---------------------------------------------------
# models = {
#     "Linear Regression": LinearRegression(),
#     "Ridge": Ridge(),
#     "Lasso": Lasso(),
#     "ElasticNet": ElasticNet(),
#     "Decision Tree": DecisionTreeRegressor(),
#     "Random Forest": RandomForestRegressor(),
#     "AdaBoost": AdaBoostRegressor(),
#     "Gradient Boosting": GradientBoostingRegressor(),
#     "LGBM": LGBMRegressor(),
#     "XGBoost": XGBRegressor(),
#     "SVM": SVR(),
#     "KNN": KNeighborsRegressor(),
#     "Bagging Regressor": BaggingRegressor()
# }


# # ---------------------------------------------------
# # Transformation Stages
# # ---------------------------------------------------
# stages = {
#     "Stage1_OneHotOnly": ColumnTransformer([
#         ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
#     ], remainder="passthrough"),

#     "Stage2_Scaling+OneHot": ColumnTransformer([
#         ("num", StandardScaler(), num_cols),
#         ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
#     ])
# }


# # ---------------------------------------------------
# # Evaluate Models at Each Stage
# # ---------------------------------------------------
# best_pipeline = None
# best_score = -999
# best_name = ""

# print("\n======================================================")
# print("MODEL EVALUATION WITH TRAIN + TEST ADJUSTED R²")
# print("======================================================")

# for stage_name, transformer in stages.items():

#     print("\n======================================================")
#     print("Running Transformation Stage:", stage_name)
#     print("======================================================")

#     for model_name, model in models.items():

#         pipeline = Pipeline([
#             ("transform", transformer),
#             ("model", model)
#         ])

#         pipeline.fit(X_train, y_train)

#         # Predictions
#         train_pred = pipeline.predict(X_train)
#         test_pred = pipeline.predict(X_test)

#         # RMSE
#         rmse_train = np.sqrt(mean_squared_error(y_train, train_pred))
#         rmse_test = np.sqrt(mean_squared_error(y_test, test_pred))

#         # R²
#         r2_train = r2_score(y_train, train_pred)
#         r2_test = r2_score(y_test, test_pred)

#         # Adjusted R²
#         adj_train = adjusted_r2(r2_train, X_train.shape[0], X_train.shape[1])
#         adj_test = adjusted_r2(r2_test, X_test.shape[0], X_test.shape[1])

#         # Overfitting Gap
#         overfit_gap = adj_train - adj_test

#         # Print Results
#         print(f"""
# Model: {model_name}
# --------------------------------------------------
# Train RMSE      : {rmse_train:.2f}
# Test RMSE       : {rmse_test:.2f}

# Train Adj R²    : {adj_train:.4f}
# Test Adj R²     : {adj_test:.4f}

# Overfit Gap     : {overfit_gap:.4f}
# --------------------------------------------------
#         """)

#         # Best Tradeoff Score
#         score = adj_test - (0.3 * overfit_gap)

#         if score > best_score:
#             best_score = score
#             best_pipeline = pipeline
#             best_name = f"{stage_name} + {model_name}"


# # ---------------------------------------------------
# # Best Model Selected
# # ---------------------------------------------------
# print("\n======================================================")
# print("✅ BEST MODEL SELECTED")
# print("======================================================")
# print("Best Model:", best_name)

# # Save Final Pipeline
# joblib.dump(best_pipeline, "movie_pipeline.pkl")
# print("\n✅ Saved Successfully: movie_pipeline.pkl")

# joblib.dump(X.columns.tolist(), "training_columns.pkl")
# print("✅ training_columns.pkl saved")

# joblib.dump(cat_cols, "categorical_cols.pkl")
# joblib.dump(num_cols, "numeric_cols.pkl")
# joblib.dump(top_categories, "top_categories.pkl")
# print("✅ Saved: top_categories.pkl")
# print("✅ Saved: categorical_cols.pkl, numeric_cols.pkl")   

#---------------------------------------------------
# second version with better structure and comments for clarity
#---------------------------------------------------

# import pandas as pd
# import numpy as np
# import joblib
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder, StandardScaler
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.ensemble import RandomForestRegressor # Example best model

# # Load Dataset
# df = pd.read_csv("bollywood_movie_data.csv")

# # 1. Drop unnecessary columns immediately
# cols_to_drop = ["Movie Name", "Release Period", "New Director", "Whether Franchise"]
# df.drop(columns=cols_to_drop, inplace=True)

# TARGET = "Revenue(INR)"
# # Use Log Transformation for better scaling (optional but recommended for money)
# y = np.log1p(df[TARGET]) 
# X = df.drop(TARGET, axis=1)

# # 2. Identify Columns AFTER dropping
# cat_cols = X.select_dtypes(include="object").columns.tolist()
# num_cols = X.select_dtypes(exclude="object").columns.tolist()

# # 3. Handle High Cardinality
# top_categories = {}
# def reduce_cardinality(df, col, top_k=15):
#     top = df[col].value_counts().nlargest(top_k).index.tolist()
#     top_categories[col] = top
#     df[col] = df[col].apply(lambda x: x if x in top else "Other")
#     return df

# high_card_cols = ["Lead Star", "Director", "Music Director"]
# for col in high_card_cols:
#     if col in X.columns:
#         X = reduce_cardinality(X, col)

# # 4. Define Pipeline
# preprocessor = ColumnTransformer([
#     ("num", StandardScaler(), num_cols),
#     ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
# ])

# # Using RandomForest as an example - replace with your best found model
# best_pipeline = Pipeline([
#     ("transform", preprocessor),
#     ("model", RandomForestRegressor(n_estimators=100, random_state=42))
# ])

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# best_pipeline.fit(X_train, y_train)

# # 5. Save everything consistently
# joblib.dump(best_pipeline, "movie_pipeline.pkl")
# joblib.dump(cat_cols, "categorical_cols.pkl")
# joblib.dump(num_cols, "numeric_cols.pkl")
# joblib.dump(top_categories, "top_categories.pkl")

# print("✅ Training Complete and Files Saved!")

#---------------------------------------------------
#third version with modular functions for better readability and maintenance
#---------------------------------------------------
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

# Load Dataset
df = pd.read_csv("bollywood_movie_data.csv")

# 1. Drop the columns you don't want in the app
cols_to_drop = ["Movie Name", "Release Period", "New Director", "Whether Franchise"]
df.drop(columns=cols_to_drop, inplace=True)

TARGET = "Revenue(INR)"
# Use log transformation because Revenue has high variance
y = np.log1p(df[TARGET]) 
X = df.drop(TARGET, axis=1)

# 2. Identify Columns AFTER dropping
cat_cols = X.select_dtypes(include="object").columns.tolist()
num_cols = X.select_dtypes(exclude="object").columns.tolist()

# 3. Handle High Cardinality AND save Genre options
top_categories = {}
def reduce_cardinality(df, col, top_k=15):
    # Get the top categories
    top = df[col].value_counts().nlargest(top_k).index.tolist()
    top_categories[col] = top
    # Replace anything else with 'Other'
    df[col] = df[col].apply(lambda x: x if x in top else "Other")
    return df

# Apply this to High Cardinality columns
for col in ["Lead Star", "Director", "Music Director"]:
    X = reduce_cardinality(X, col)

# Save ALL unique values for Genre (since it's not high cardinality, we keep all)
top_categories["Genre"] = X["Genre"].unique().tolist()

# 4. Define and Train Pipeline
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

pipeline = Pipeline([
    ("transform", preprocessor),
    ("model", RandomForestRegressor(n_estimators=100, random_state=42))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

# 5. Save everything
joblib.dump(pipeline, "movie_pipeline.pkl")
joblib.dump(cat_cols, "categorical_cols.pkl")
joblib.dump(num_cols, "numeric_cols.pkl")
joblib.dump(top_categories, "top_categories.pkl")

print("✅ Training finished. Files saved successfully.")



✅ Training finished. Files saved successfully.
